In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tensorflow as tf

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [3]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
API_KEY = os.environ.get('API_KEY')

In [39]:
BIGdf = pd.read_csv('comments/comments95_110.csv')

In [41]:
BIGdf.value_counts('video_id').keys()

Index(['-AuJiwjsmWk', '-BCwcr606g0', '-ARdLKWyGfA', '-B3Q9t8pgEA'], dtype='object', name='video_id')

In [10]:
likes = np.nan

In [11]:
def making_weights(num):
    '''This function makes weights for each comment based on its like count (num)'''
    if num == 0:
        return 1
    elif num > 0 and num <= np.median(likes[:len(likes)//2]):
        return 2
    elif num > np.median(likes[:len(likes)//2]) and num <= np.median(likes):
        return 3
    elif num > np.median(likes) and num < np.median(likes[len(likes)//2:]):
        return 4
    else:
        return 5

In [12]:
def df_cutter(df):
    IDs_list = df.value_counts('video_id').keys()
    cut_dfs = []
    for i, video in enumerate(IDs_list):
        cut_df = df[df['video_id'] == IDs_list[i]]
        global likes
        likes = sorted(list(cut_df[cut_df['likecount'] > 0]['likecount']))
        cut_df['weight'] = cut_df['likecount'].apply(making_weights)
        cut_dfs.append(cut_df)
    return cut_dfs

In [42]:
cut_dfs = df_cutter(BIGdf)

/var/folders/lp/kbcy9lh91kjc0lpdp8ydq1wr0000gn/T/ipykernel_71553/2854793023.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cut_df['weight'] = cut_df['likecount'].apply(making_weights)
/var/folders/lp/kbcy9lh91kjc0lpdp8ydq1wr0000gn/T/ipykernel_71553/2854793023.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cut_df['weight'] = cut_df['likecount'].apply(making_weights)
/var/folders/lp/kbcy9lh91kjc0lpdp8ydq1wr0000gn/T/ipykernel_71553/2854793023.py:8: SettingWithCopyWarning: 
A value is trying to be set

In [43]:
cut_dfs[1]

,Unnamed: 0,comment,author,likecount,date,replies,comment_clean,video_id,weight
399,0,A 100ft reel of 16mm film was harmed in the ma...,The Slow Mo Guys,4615,2021-02-11T15:50:33Z,84,0 a 100ft reel of 16mm film was harmed in t...,-BCwcr606g0,5
400,1,I've been an analogue motion film camera techn...,Paul Scaglione,264,2021-02-22T16:05:53Z,0,0 ive been an analogue motion film camera t...,-BCwcr606g0,5
401,2,This just made me love film so much more,Steezy Kane,68,2021-05-29T08:12:27Z,1,0 this just made me love film so much more\...,-BCwcr606g0,4
402,3,What I find most fascinating is that claw pull...,Dream XCVIII,126,2021-04-13T00:08:53Z,3,0 what i find most fascinating is that claw...,-BCwcr606g0,5
403,4,This is awesome. Love watching the mechanics,TimeBucks,969,2021-02-12T09:38:21Z,3,0 this is awesome love watching the mechani...,-BCwcr606g0,5
...,...,...,...,...,...,...,...,...,...
579,180,"wow, thanks for this, I've got education in ph...",peggyloveoscar,0,2021-02-12T22:01:04Z,0,0 wow thanks for this ive got education in ...,-BCwcr606g0,1
580,181,Perhaps my favorite of your videos so far. Th...,Noel Davis,0,2021-02-17T20:24:22Z,0,0 perhaps my favorite of your videos so far...,-BCwcr606g0,1
581,182,The engineering of film cameras is amazing.\nT...,CB Du Rietz,0,2021-02-16T20:15:52Z,0,0 the engineering of film cameras is amazin...,-BCwcr606g0,1
582,183,"This is so cool, and I feel smarter just from ...",Anna Zverina,3,2021-02-11T15:49:14Z,0,0 this is so cool and i feel smarter just f...,-BCwcr606g0,3


In [16]:
def sentiment_score_comment(df):

    '''This function predicts the sentiment score of each youtube video!'''
    
    model_name = "cardiffnlp/twitter-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

    # Lists to store the sentiment analysis results
    sentiment_list = []
    negative_list = []
    neutral_list = []
    positive_list = []
    scalar_value_list = []
    weighted_SV = []
    weight = list(df['likecount'].apply(making_weights))

    # Iterate over the comments in the DataFrame
    for i, text in enumerate(df['comment']):
        
        # Tokenization, Sentiment Prediction, and Interpretation
        tokens = tokenizer.encode_plus(text, add_special_tokens=True, padding='longest', truncation=True, max_length=512, return_tensors='tf')
        outputs = model(tokens.input_ids)
        logits = outputs.logits
        prediction = np.array(tf.nn.softmax(logits)[0])
        predicted_class = tf.argmax(logits, axis=1).numpy()[0]
        sentiment_labels = ["Negative", "Neutral", "Positive"]
        predicted_sentiment = sentiment_labels[predicted_class]

        # Append the sentiment analysis results to the respective lists
        sentiment_list.append(predicted_sentiment)
        negative_list.append(round(prediction[0]*100, 2))
        neutral_list.append(round(prediction[1]*100, 2))
        positive_list.append(round(prediction[2]*100, 2))
        scalar_value_val = round((prediction[0])*-1+(prediction[2]*1),2)
        scalar_value_list.append(scalar_value_val)
        weighted_SV.append(df['weight'].iloc[i] * scalar_value_val)


    # Create a new DataFrame with the sentiment analysis results
    results_df = pd.DataFrame({
        'Comment': df['comment'],
        'Sentiment': sentiment_list,
        'Negative (%)': negative_list,
        'Neutral (%)': neutral_list,
        'Positive (%)': positive_list,
        'Scaler_value': scalar_value_list,
        'weighted_SV': weighted_SV,
        'weight': weight
    })

    # Return the new DataFrame
    return results_df


In [28]:
results_list = []
for df in cut_dfs:
    results = sentiment_score_comment(df)
    results_list.append(results)
results_list

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the mode

[                                               Comment Sentiment  \
 112  "there's nothing wrong with it being bent"\nI ...   Neutral   
 113  So a unripened apple is a sweet potato and a r...  Positive   
 114  This is just 17 minutes of Rhett and link disc...  Positive   
 115  I love how Link  has a song for almost every a...  Positive   
 116  The angles Link chooses to feed Rhett cracks m...  Negative   
 ..                                                 ...       ...   
 294  *Link character info*\n\n*Likes* - dipping car...   Neutral   
 295  I live near your uncle's farm, Rhett! That's s...  Positive   
 296  The way rhett tells stories reminds me of edwa...   Neutral   
 297  Link does realize that a loss of / inhibited a...  Negative   
 298                       “Rabbits don’t have hummus.”  Negative   
 
      Negative (%)  Neutral (%)  Positive (%)  Scaler_value  weighted_SV  \
 112         10.61        67.02         22.37          0.12         0.60   
 113          5.72

In [31]:
results_list[3]

,Comment,Sentiment,Negative (%),Neutral (%),Positive (%),Scaler_value,weighted_SV,weight
299,R.I.P. Mike Howe\nR.I.P. Joey Jordison\nR.I.P....,Negative,71.03,25.91,3.06,-0.68,-3.40,5
300,Being in my mid 60's this type of sad news in ...,Negative,57.96,31.50,10.54,-0.47,-2.35,5
301,"Nobody could sing “Tush” or “I got the Six, Gi...",Neutral,23.21,44.86,31.93,0.09,0.45,5
302,"Man, way too many badass musicians are dying l...",Negative,57.53,32.07,10.40,-0.47,-1.88,4
303,"RIP Dusty, you gave us some of the finest soun...",Positive,0.37,3.62,96.01,0.96,1.92,2
...,...,...,...,...,...,...,...,...
394,"Oh no, heartbreaking news. Sympathy to his fam...",Positive,7.26,23.93,68.81,0.62,0.62,1
395,A Texas blues / rock legend. My dad has seen z...,Positive,0.90,23.71,75.39,0.74,2.22,3
396,50 years from now we will all be blasting ZZ T...,Positive,0.42,16.14,83.43,0.83,0.83,1
397,Man what a sad and heart breaking week for the...,Negative,93.86,5.43,0.71,-0.93,-0.93,1


In [46]:
IDs_df = pd.DataFrame(BIGdf.value_counts('video_id').keys())

In [47]:
IDs_df

,video_id
0,-AuJiwjsmWk
1,-BCwcr606g0
2,-ARdLKWyGfA
3,-B3Q9t8pgEA


In [48]:
IDs_df['positivity_score'] = np.nan

In [53]:
IDs_df

,video_id,positivity_score
0,-AuJiwjsmWk,NaN
1,-BCwcr606g0,NaN
2,-ARdLKWyGfA,NaN
3,-B3Q9t8pgEA,NaN


In [54]:
for i, df in enumerate(cut_dfs):
    
    df['weight'] = df['likecount'].apply(making_weights)
    
    score_df = sentiment_score_comment(df)
    
    positivity_score = score_df['weighted_SV'].mean()
    
    IDs_df['positivity_score'][i] = positivity_score

/var/folders/lp/kbcy9lh91kjc0lpdp8ydq1wr0000gn/T/ipykernel_71553/4242844407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = df['likecount'].apply(making_weights)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/var/folders/lp/kbcy9lh91kjc0lpdp8ydq1wr0000gn/T/ipykernel_71553/4242844407.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [40]:
results = sentiment_score_comment(df)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [56]:
IDs_df

,video_id,positivity_score
0,-AuJiwjsmWk,0.571711
1,-BCwcr606g0,1.472757
2,-ARdLKWyGfA,1.850804
3,-B3Q9t8pgEA,0.558800


In [57]:
def df_to_csv(results_df):
    IDs_df.to_csv('video_score.csv')
    
    

In [61]:
df_to_csv(IDs_df)